In [ ]:
!pip install autoPyTorch[forecasting]

In [59]:
from autoPyTorch.api.time_series_forecasting import TimeSeriesForecastingTask
import pandas as pd

In [53]:
data = pd.read_csv('covid_deaths_dataset.csv').set_index('date')

targets = data[['T1']]
#features = data[[col for col in data.columns if col != 'T266']]

In [54]:
forecasting_horizon = 5

y_train = [targets[: -forecasting_horizon]]
y_test = [targets[-forecasting_horizon:]]

#X_train = [features[: -forecasting_horizon]]
#known_future_features = list(features.columns)
#X_test = [features[-forecasting_horizon:]]

In [55]:
api = TimeSeriesForecastingTask(seed=0, n_jobs=5, 
    #output_directory='model', delete_output_folder_after_terminate=False, 
    ensemble_size=10, ensemble_nbest=10)

api.search(
    y_train=y_train,
    optimize_metric='mean_MAPE_forecasting',
    n_prediction_steps=forecasting_horizon,
    #memory_limit=16 * 1024,  # Currently, forecasting models use much more memories
    func_eval_time_limit_secs=500,
    total_walltime_limit=1000
    #min_num_test_instances=1000,  # proxy validation sets. This only works for the tasks with more than 1000 series
    #known_future_features=known_future_features,
)

In [56]:
test_sets = api.dataset.generate_test_seqs()
y_pred = api.predict(test_sets)
print(y_test, y_pred)

[              T1
date            
2020-08-11  1344
2020-08-12  1354
2020-08-13  1363
2020-08-14  1363
2020-08-15  1370
2020-08-16  1375
2020-08-17  1375
2020-08-18  1375
2020-08-19  1375
2020-08-20  1385] [[1328. 1328. 1328. 1328. 1328. 1328. 1328. 1328. 1328. 1328.]]


In [67]:
predictions_df = pd.read_csv('covid_deaths_dataset_T1.csv', index_col=0)

In [68]:
predictions_df.loc['autoPyTorch'] = y_pred[0]

,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20
dummy,1370.000000,1370.000000,1370.000000,1370.000000,1370.000000
autoregression,1375.007728,1381.086613,1385.975523,1394.794212,1402.769550
moving average,1342.800000,1349.600000,1355.900000,1362.200000,1367.900000
ARMA,1375.615483,1380.982707,1386.066401,1390.904488,1395.461852
TBATS,1377.138883,1384.258050,1391.377217,1398.496385,1405.615552
autoPyTorch,1328.000000,1328.000000,1328.000000,1328.000000,1328.000000


In [ ]:
predictions_df.to_csv('covid_deaths_dataset_T1.csv')